In [ ]:
# Подгружаем необходимые библиотеки для анализа данных и визуализации

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Вам нужно форкнуть скрипт fetch, чтобы можно было его импортировать. Затем нужно File>Add Utility Script>fetch

import fetch

In [ ]:
# Используем скрипт от grwlf чтобы скачать последние данные с яндекса. Будем это делать в 2 ячейки:

data = fetch.fetch_yandex(dump_folder='')

In [ ]:
# Вторая ячейка, данные сохранятся в файл с именем времени скачивания. Запомним имя в переменную.

data, filepath = fetch.format_csse2(data, dump_folder='')

In [ ]:
# Считываем исторические данные

russia = pd.read_csv('https://raw.githubusercontent.com/grwlf/COVID-19_plus_Russia/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_RU.csv')

In [ ]:
# Сфетченные данные считываем в нужном формате

russia_latest = pd.read_csv(filepath)
russia_latest.head()

In [ ]:
russia.iloc[0,6]
#.loc[0,'Province_State']

In [ ]:
# Исторические

russia.iloc[:,-3:].head()

In [ ]:
# Собираем все данные

rus = russia.set_index('Province_State').join(russia_latest.set_index('Province_State')['Confirmed'])

In [ ]:
rus.head()

In [ ]:
# Вычисляем сегодня из имени файла

today = filepath[:10]
today2 = today[3:5]+'/'+today[:2]+'/'+today[-2:]

In [ ]:
# Переименовываем и убираем лишние колонки

rus.drop(['UID','iso2','iso3','FIPS','Admin2','Country_Region','Lat','Long_','Combined_Key','code3'], axis=1, inplace=True)
rus[today2] = rus['Confirmed']
del rus['Confirmed']

In [ ]:
# Все данные есть:

rus

In [ ]:
# Данные пока не обновились - обновление в 14:30 по НСК примерно

rus['06/02/20'] = rus['06/03/20']
del rus['06/03/20']

In [ ]:
df = rus.T.iloc[-30:,:]
df

In [ ]:
# Посмотрим на последние 30 дней

df.plot(figsize=(15,10), legend=None)

plt.show()

In [ ]:
# Лучше взять логарифм, чтобы не смотреть только на Москву. Логарифма от 0 нет, поэтому добавляем 0.5

(np.log(rus + 0.5).T).plot(figsize=(15,10), legend=None)

plt.show()

Все кривые выглядят схоже, поэтому я возьму средние значение и предположу, что они продолжатся для Новосибирской области.

In [ ]:
# А что если бы всех поставить с 148+ случаями в начало графика?

y = np.log(rus + 0.5).T
y_m = y[y>5]
print ('Лог шкала:', [(one, np.round(np.exp(one),0)) for one in range(12)])

In [ ]:
y_m

In [ ]:
y_m.count().max()

In [ ]:
list_columns = []

for column in y_m.columns:
    list_columns.append(y_m[column].count())
    
filled = pd.Series(list_columns).max()

print('Макс кол-во наблюдений', filled)

In [ ]:
# Пустой датафрейм

y_gt_148 = pd.DataFrame(data = [[0 for i in range(len(y_m.columns))] for j in range(filled)], \
                        index = range(filled), columns = y_m.columns)

y_gt_148

In [ ]:
# Наполним датафрейм с первого наблюдения, где 148+ кейсов в логарифмической шкале

for i in range(len(y_m.columns)):
    temp = y_m.iloc[:,i].dropna().reset_index(drop=True)
    #print(temp)
    y_gt_148.iloc[:temp.shape[0],i] = temp

In [ ]:
y_gt_148 = y_gt_148.replace(0,np.nan)

In [ ]:
y_gt_148

In [ ]:
# Пусть будет любое количество случаев, все вначале.

y_gt_148.plot(figsize=(15,10), legend=None)

plt.show()

In [ ]:
# Давайте все линии начнем в одной точке

# Самое большое значение для первого наблюдения

y_gt_148.iloc[0,:].max()

In [ ]:
# Подравняем все наблюдения на полученный максимум

delta = y_gt_148.iloc[0,:].max() - y_gt_148.iloc[0,:]
all_in_one = y_gt_148 + delta
all_in_one.head()

In [ ]:
# Пусть будут все исходить из одной точки.

all_in_one.plot(figsize=(15,10), legend=None)

plt.show()

# Давайте сравним Новосибирск с Москвой и другими схожими регионами. 

In [ ]:
# Считаем список регионов для которых нужно предсказывать

russia_regions = pd.read_csv('/kaggle/input/russia-regions-in-sber-covid-competition/russia_regions.csv')

In [ ]:
russia_regions.head()

In [ ]:
# Манипуляции, чтобы выйти на нужный формат

# Автономные области не названы, поможем назвать

russia_regions.loc[russia_regions['iso_code'] == 'RU-NEN', 'csse_province_state'] = 'Nenetskiy autonomous oblast'
russia_regions.loc[russia_regions['iso_code'] == 'RU-CHU', 'csse_province_state'] = 'Chukotskiy autonomous oblast'

# Привести в соответствие с конкурсным перечнем регионов

rus['ind'] = rus.index
# Altay republic > Republic of Altay
rus.loc[rus.index == 'Altay republic', 'ind'] = 'Republic of Altay'
rus.set_index('ind', inplace=True)

In [ ]:
sorted_obl[sorted_obl['name']=='Новосибирская'].index[0]

In [ ]:
# Найдем области, похожие на Новосибирскую по населению

sorted_obl = russia_regions.sort_values(['population']).reset_index()

nsk_index = sorted_obl[sorted_obl['name']=='Новосибирская'].index[0]

sorted_obl[nsk_index-2:nsk_index+3]

In [ ]:
selected_regions = list(sorted_obl[nsk_index-2:nsk_index+3]['csse_province_state'])

In [ ]:
selected_regions

In [ ]:
show_regions = selected_regions + ['Moscow']
all_in_one[show_regions]

In [ ]:
all_in_one[show_regions].plot(figsize=(15,10))
plt.show()

In [ ]:
(np.log(rus.loc[show_regions] + 0.5).T[-30:]).plot(figsize=(15,10))
plt.show()

In [ ]:
last_30_days = np.log(rus.loc[show_regions] + 0.5).T[-30:]
last_30_days.head()

In [ ]:
last_30_days = last_30_days - last_30_days.loc['05/03/20']
last_30_days.head()

In [ ]:
last_30_days.plot(figsize=(15,10))

plt.show()

In [ ]:
last_100_days = np.log(rus.loc[show_regions] + 0.5).T[-100:]
#last_100_days = last_30_days - last_30_days.loc['05/03/20']
last_100_days.plot(figsize=(15,10))
plt.show()

In [ ]:
d100 = last_100_days[last_100_days > 5]

filled2 = d100.count().max()

In [ ]:

d100_eq = pd.DataFrame(data = [[0 for i in range(len(d100.columns))] for j in range(filled2)], \
                        index = range(filled2), columns = d100.columns)

# Наполним датафрейм с первого наблюдения, где 148+ кейсов в логарифмической шкале

for i in range(len(d100.columns)):
    temp = d100.iloc[:,i].dropna().reset_index(drop=True)
    d100_eq.iloc[:temp.shape[0],i] = temp
    
d100_eq = d100_eq.replace(0,np.nan)

print(d100_eq.iloc[0,:].max())

delta = d100_eq.iloc[0,:].max() - d100_eq.iloc[0,:]
d100_eq = d100_eq + delta

print(d100_eq.head())

d100_eq.plot()
plt.show()

In [ ]:
d100.plot()
plt.show()

In [ ]:
d100

In [ ]:
d100['Novosibirsk oblast'].dropna().plot(figsize=(15,10))
plt.show()

In [ ]:
nsk = rus.loc['Novosibirsk oblast']
nsk

In [ ]:
nsk['06/01/20'] = 2914
nsk['06/02/20'] = 3020

In [ ]:
nsk.tail()

In [ ]:
del nsk['06/03/20']

In [ ]:
nsk['06/03/20'] = 3122  

In [ ]:
nsk.tail()

In [ ]:
nsk[nsk>150].plot(figsize=(15,10))

In [ ]:
delta_nsk = nsk - nsk.shift()

delta_nsk = delta_nsk[delta_nsk>0]

In [ ]:
delta_nsk.plot(figsize=(15,10))
plt.show()

In [ ]:
log_nsk = np.log(nsk)
log_nsk = log_nsk[log_nsk>5]

In [ ]:
log_nsk.plot(figsize=(15,10))
plt.show()

In [ ]:
log_nsk.shape

In [ ]:
df_nsk = pd.DataFrame(data = {'date' : pd.to_datetime(log_nsk.index), 'Nsk' : log_nsk.values, 'X' : range(1,44)})

df_nsk.tail(3)

In [ ]:
df_nsk.head(3)

In [ ]:
# Учимся на 10 днях и предсказываем на 30
# Модель y = x^a или ln y = a * ln x

X_train = np.log(df_nsk.loc[0:9,'X']).values.reshape(-1,1) 
y_train = np.log(df_nsk.loc[0:9,'Nsk']).values.reshape(-1,1)

X_test = np.log(df_nsk.loc[10:39,'X']).values.reshape(-1,1) 
y_test = np.log(df_nsk.loc[10:39,'Nsk']).values.reshape(-1,1) 


In [ ]:
X_train, y_train

In [ ]:
# Сделаем скользящим окном

for day in range(1,4):

    X_train = np.log(df_nsk.loc[0 : 9 + day,'X']).values.reshape(-1,1) 
    y_train = np.log(df_nsk.loc[0 : 9 + day,'Nsk']).values.reshape(-1,1)

    X_test = np.log(df_nsk.loc[10 + day : 39 + day,'X']).values.reshape(-1,1) 
    y_test = np.log(df_nsk.loc[10 + day : 39 + day,'Nsk']).values.reshape(-1,1) 

    # Скалируем признаки, для регрессии с регуляризацией хорошо
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_train)

    # print(scaler.mean_, scaler.var_)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Строим регрессию
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X_train_scaled, y_train)

    # Функции для 2 метрик: первая - конкурсная, вторая - процент отклонения от истины
    def MALE(pred, true):
    #    print(np.log10((pred + 1) / (true + 1)))
        return np.mean(np.abs(np.log10((pred + 1) / (true + 1))))

    def AvgProc(pred, true):
    #    print((pred-true)/true)
        return np.mean(np.abs((pred-true)/true))

    # Приводим у к кол-ву случаев
    y_pred_test_exp = np.round(np.exp(np.exp(reg.predict(X_test_scaled))),0)
    y_pred_train_exp = np.round(np.exp(np.exp(reg.predict(X_train_scaled))),0)
    y_test_exp = np.round(np.exp(np.exp(y_test)),0)
    y_train_exp = np.round(np.exp(np.exp(y_train)),0)

    # Выводим результаты
    print('coefs=', [(name,value) for (name,value) in zip(['ln NSK'],reg.coef_)], 'const=', reg.intercept_)
    print('MALE test = ', MALE(y_pred_test_exp[:y_test.shape[0]], y_test_exp), 'MALE train =', MALE(y_pred_train_exp, y_train_exp))
    print('AvgProc test = ', AvgProc(y_pred_test_exp[:y_test.shape[0]], y_test_exp), 'AvgProc train =', AvgProc(y_pred_train_exp, y_train_exp))


    plt.figure(figsize=(15,10))

    plt.plot(df_nsk.loc[10 + day : 39 + day,'date'], y_pred_test_exp) 
    plt.plot(df_nsk.loc[10 + day : 39 + day,'date'], y_test_exp)
    plt.plot(df_nsk.loc[0 : 9 + day,'date'], y_pred_train_exp)
    plt.plot(df_nsk.loc[0 : 9 + day,'date'], y_train_exp)

    plt.legend()
    plt.legend()
    plt.show()


# Далее код из похожей задачи, может пригодиться

In [ ]:
# # i это индекс, который будет меняться по регионам: берем первую область (край и т.п.)

# scores = []
# percent = []

# for i in range(len(rus.T.columns)):

#     obl = pd.DataFrame(rus.T.iloc[:,i].copy(), columns=[rus.T.iloc[:,i].name]) 
    
#     # Исправим данные, где кол-во растет, потом падает: пусть только растет
#     prev = 0
#     obl_corrected = []

#     for one in obl.iloc[:,0]:
#         if one < prev:
#             obl_corrected[-1] = one

#         obl_corrected.append(one)  
#         prev = one

#     obl.iloc[:,0] = obl_corrected

#     # Индекс приводим к формату даты
#     obl.index = pd.to_datetime(obl.index) 

#     ## Делаем признаки

#     # Порядковый номер дня в году - "дата" для всех
#     obl['day_of_year'] = obl.index.dayofyear 

#     # Создаем day7 - на 1 возрастают дни (от начала года) со времени, когда кейсов стали больше 7 (после какого-то значения рост более стабильный)
#     obl['day7'] = 0 
#     idx_day_7 = obl.iloc[:,0]>7
#     obl.loc[idx_day_7,'day7'] = obl.loc[idx_day_7,'day7'].index.dayofyear
#     day_day7 = obl.loc[obl['day7'] > 0, 'day7'].min() - 1
#     obl.loc[obl['day7'] > 0, 'day7'] -= day_day7 

#     # Создаем apr6 - на 1 возрастают дни с 6 апреля - когда должны заметить карантин (30.03+7 дней) - траектория изменилась
#     obl['apr6'] = 0 
#     idx_apr_6 = obl.index >= '2020-04-06'
#     obl.loc[idx_apr_6,'apr6'] = obl.loc[idx_apr_6,'apr6'].index.dayofyear
#     day_apr6 = obl.loc[obl['apr6'] > 0, 'apr6'].min() - 1
#     obl.loc[obl['apr6'] > 0, 'apr6'] -= day_apr6 
    
# #     # Создаем last7 - на 1 возрастают дни для последних 7 дней - больше учитываем последние данные
# #     obl['last7'] = 0 
# #     idx_last_7 = obl.index >= obl.index[-11]
# #     obl.loc[idx_last_7,'last7'] = obl.loc[idx_last_7,'last7'].index.dayofyear
# #     day_last7 = obl.loc[obl['last7'] > 0, 'last7'].min() - 1
# #     obl.loc[obl['last7'] > 0, 'last7'] -= day_last7
    
# #     # И сделаем квадрат от last7
# #     obl['last7_2'] = obl['last7'] ** 2

#     pos_cases = obl.iloc[:,0]>0
    
#     # Давайте посмотрим на 2 режима - day7 (сверху) и apr6. Что видно?
#     plt.figure(figsize=(15,10))
#     plt.plot(np.log(obl.loc[pos_cases].iloc[:,0]))
#     plt.scatter(obl.loc[pos_cases,'day7'].index,(obl.loc[pos_cases,'day7']>0)+0.1, label='day7')
#     plt.scatter(obl.loc[pos_cases,'apr6'].index,(obl.loc[pos_cases,'apr6']>0), label='apr6')
#     plt.title(obl.columns[0])
#     plt.legend()
#     plt.show()
    
# #     # Собираем Х
# #     X_cols = ['day_of_year','day7','apr6','last7', 'last7_2']
    
# #     # до 26 обучаем, c 27 по 03 будем тестироваться
# #     train = obl.loc[(pos_cases) & (obl.index<'2020-04-27')]    
    
# #     wk3 = ['2020-04-27','2020-04-28','2020-04-29','2020-04-30','2020-05-01','2020-05-02','2020-05-03']
# #     test  = pd.DataFrame(index = pd.to_datetime(wk3), data = [], columns = X_cols)
# #     test['day_of_year'] = test.index.dayofyear
# #     count_down = test.index.dayofyear - test.index.dayofyear.min()
# #     test['day7'] = count_down + train['day7'].max() + 1
# #     test['apr6'] = count_down + train['apr6'].max() + 1
# #     test['last7'] = count_down + 8
# #     test['last7_2'] = test['last7'] ** 2

# #     # Х в лог
# #     X_train = np.log(train[X_cols] + 0.5)
# #     y_train = np.log(train.iloc[:,0] + 0.5)
# #     X_test = np.log(test[X_cols] + 0.5)

# #     y_test  = obl.loc[obl.index>'2020-04-26',obl.columns[0]]
# #     y_test = np.log(y_test + 0.5)

# #     # Скалируем признаки, для регрессии с регуляризацией хорошо
# #     from sklearn.preprocessing import StandardScaler
# #     scaler = StandardScaler()
# #     scaler.fit(X_train)
# #     # print(scaler.mean_, scaler.var_)
# #     X_train_scaled = scaler.transform(X_train)
# #     X_test_scaled = scaler.transform(X_test)

# #     # Строим регрессию
# #     from sklearn.linear_model import LinearRegression
# #     reg = LinearRegression().fit(X_train_scaled, y_train.values)

# #     # Функции для 2 метрик: первая - конкурсная, вторая - процент отклонения от истины
# #     def MALE(pred, true):
# #     #    print(np.log10((pred + 1) / (true + 1)))
# #         return np.mean(np.abs(np.log10((pred + 1) / (true + 1))))

# #     def AvgProc(pred, true):
# #     #    print((pred-true)/true)
# #         return np.mean(np.abs((pred-true)/true))

# #     # Приводим у к кол-ву случаев
# #     y_pred_test_exp = np.round(np.exp(reg.predict(X_test_scaled))-0.5,0)
# #     y_pred_train_exp = np.round(np.exp(reg.predict(X_train_scaled))-0.5,0)
# #     y_test_exp = np.round(np.exp(y_test)-0.5,0)
# #     y_train_exp = np.round(np.exp(y_train)-0.5,0)

# #     # Обрабатываем "Архангельский прыжок" (резкое увеличение случаев - увеличиваем предсказ на 2/3 от скачка)
# #     if i==2: 
# #         y_pred_test_exp = y_pred_test_exp + (y_train_exp[-1]-y_train_exp[-2])*(2/3)
        
# #     # Выводим результаты
# #     print(obl.columns[0])
# #     print('coefs=', [(name,value) for (name,value) in zip(X_cols,reg.coef_)], 'const=', reg.intercept_)
# #     print('MALE test = ', MALE(y_pred_test_exp[:y_test.shape[0]], y_test_exp), 'MALE train =', MALE(y_pred_train_exp, y_train_exp))
# #     print('AvgProc test = ', AvgProc(y_pred_test_exp[:y_test.shape[0]], y_test_exp), 'AvgProc train =', AvgProc(y_pred_train_exp, y_train_exp))

# #     # Отрисуем предсказ
# #     plt.figure(figsize=(15,10))
    
# #     wk3dt = pd.to_datetime(wk3)
# #     for x,y in zip(y_test.index, y_test_exp):
# #         label = "{:.0f}".format(y)
# #         plt.annotate(label, # this is the text
# #                      (x,y), # this is the point to label
# #                      textcoords="offset points", # how to position the text
# #                      xytext=(0,-15), # distance from text to points (x,y)
# #                      ha='center') # horizontal alignment can be left, right or center

# #     for x,y in zip(wk3dt, y_pred_test_exp):
# #         label = "{:.0f}".format(y)
# #         plt.annotate(label, # this is the text
# #                      (x,y), # this is the point to label
# #                      textcoords="offset points", # how to position the text
# #                      xytext=(0,-15), # distance from text to points (x,y)
# #                      ha='center') # horizontal alignment can be left, right or center

# #     plt.title(obl.columns[0]+' predict')

# #     plt.plot(wk3dt, y_pred_test_exp, 'ro-', label='pred')
# #     plt.plot(y_test.index, y_test_exp, 'bo-', label='fact')

# #     plt.legend()
# #     plt.grid()
# #     plt.show()

# #     # Отрисуем модель
# #     plt.figure(figsize=(15,10))

# #     for x,y in zip(y_train.index, y_train_exp):
# #         label = "{:.0f}".format(y)
# #         plt.annotate(label, # this is the text
# #                      (x,y), # this is the point to label
# #                      textcoords="offset points", # how to position the text
# #                      xytext=(0,-15), # distance from text to points (x,y)
# #                      ha='center') # horizontal alignment can be left, right or center

# #     for x,y in zip(y_train.index, y_pred_train_exp):
# #         label = "{:.0f}".format(y)
# #         plt.annotate(label, # this is the text
# #                      (x,y), # this is the point to label
# #                      textcoords="offset points", # how to position the text
# #                      xytext=(0,-15), # distance from text to points (x,y)
# #                      ha='center') # horizontal alignment can be left, right or center

# #     plt.title(obl.columns[0]+' fit')

# #     plt.plot(y_train.index, y_pred_train_exp, 'ro-', label='pred')
# #     plt.plot(y_train.index, y_train_exp, 'bo-', label='fact')

# #     plt.legend()
# #     plt.grid()
# #     plt.show()
    
# #     scores.append((obl.columns[0], MALE(y_pred_test_exp[:y_test.shape[0]], y_test_exp)))
# #     percent.append((obl.columns[0], AvgProc(y_pred_test_exp[:y_test.shape[0]], y_test_exp)))
    

In [ ]:
# errors = pd.DataFrame([(one[0],one[1],two[1]) for (one,two) in zip(scores,percent)], columns=['Region','MALE','percent'])

In [ ]:
# pd.set_option('display.max_rows', None)
# print (errors.sort_values('MALE', ascending=False))



In [ ]:
# pd.set_option('display.max_rows', 10)
# print (pd.DataFrame(scores).sort_values([1], ascending=False))

In [ ]:
# SCORE Avg MALE

# np.mean([two for (one,two) in scores]), np.mean([two for (one,two) in percent])